In [1]:
import pickle
import numpy as np
from keras.models import Model
from keras.layers import Input, Lambda, Reshape, LSTM, Dense
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [2]:
tokenized_text, char_to_idx = pickle.load(open('../data/processed-alice.pickle', 'rb'))
X, Y = pickle.load(open('../data/alice-x-y-data.pickle', 'rb'))

Y = np.reshape(Y, (X.shape[1], X.shape[0], X.shape[2]))

# Global Variables
m, Tx, n_values = X.shape
n_a = 64 # for LSTM with 64-dimensional hidden states

# Train the Model
First we train the model to get the proper weights.
Remember, we must declare the layers in our model as global variables so they
can be reused in the Text Generation step

In [3]:
# Reused Layer for Text Generation: Reshape, LSTM, Dense
reshapor = Reshape((1, n_values))
LSTM_cell = LSTM(n_a, return_state=True)
densor = Dense(n_values, activation='softmax')

In [4]:
def learn_to_talk(Tx, n_a, n_values):
    X = Input(shape=(Tx, n_values))
    
    # Define initial hidden state for LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    outputs = []
    for t in range(Tx):
        # Get t'th vector
        x = Lambda(lambda x: x[:,t,:])(X)
        # Reshape x to be (1,Tx)
        x = reshapor(x)
        # Perform 1 step of LSTM
        a, _, c = LSTM_cell(x)
        # apply densor to hidden state output of LSTM_cell
        out = densor(a)
        outputs.append(out)
    return Model([X, a0, c0], outputs)

In [5]:
model = learn_to_talk(Tx, n_a, n_values)

In [6]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

In [12]:
model.fit([X, a0, c0], list(Y), epochs=100)

Epoch 1/100
973/973 [==============================] - 7s 7ms/step - loss: 382.0642 - dense_1_loss: 3.2631 - dense_1_acc: 0.1675 - dense_1_acc_1: 0.3864 - dense_1_acc_2: 0.4388 - dense_1_acc_3: 0.4430 - dense_1_acc_4: 0.4018 - dense_1_acc_5: 0.1398 - dense_1_acc_6: 0.1336 - dense_1_acc_7: 0.0185 - dense_1_acc_8: 0.1829 - dense_1_acc_9: 0.2600 - dense_1_acc_10: 0.2539 - dense_1_acc_11: 0.4080 - dense_1_acc_12: 0.4512 - dense_1_acc_13: 0.5087 - dense_1_acc_14: 0.3022 - dense_1_acc_15: 0.2354 - dense_1_acc_16: 0.2662 - dense_1_acc_17: 0.2138 - dense_1_acc_18: 0.2929 - dense_1_acc_19: 0.1850 - dense_1_acc_20: 0.2641 - dense_1_acc_21: 0.3823 - dense_1_acc_22: 0.3566 - dense_1_acc_23: 0.0216 - dense_1_acc_24: 0.3237 - dense_1_acc_25: 0.2785 - dense_1_acc_26: 0.2169 - dense_1_acc_27: 0.1829 - dense_1_acc_28: 0.3977 - dense_1_acc_29: 0.5190 - dense_1_acc_30: 0.3309 - dense_1_acc_31: 0.3392 - dense_1_acc_32: 0.1429 - dense_1_acc_33: 0.2302 - dense_1_acc_34: 0.4676 - dense_1_acc_35: 0.2939 - den

 64/973 [>.............................] - ETA: 6s - loss: 381.1306 - dense_1_loss: 3.0030 - dense_1_acc: 0.1875 - dense_1_acc_1: 0.3906 - dense_1_acc_2: 0.4531 - dense_1_acc_3: 0.4062 - dense_1_acc_4: 0.4531 - dense_1_acc_5: 0.1094 - dense_1_acc_6: 0.2188 - dense_1_acc_7: 0.0156 - dense_1_acc_8: 0.1094 - dense_1_acc_9: 0.3438 - dense_1_acc_10: 0.3281 - dense_1_acc_11: 0.5000 - dense_1_acc_12: 0.5469 - dense_1_acc_13: 0.5156 - dense_1_acc_14: 0.4375 - dense_1_acc_15: 0.1250 - dense_1_acc_16: 0.2812 - dense_1_acc_17: 0.2344 - dense_1_acc_18: 0.2344 - dense_1_acc_19: 0.1875 - dense_1_acc_20: 0.1719 - dense_1_acc_21: 0.4375 - dense_1_acc_22: 0.2969 - dense_1_acc_23: 0.0312 - dense_1_acc_24: 0.2812 - dense_1_acc_25: 0.2031 - dense_1_acc_26: 0.1406 - dense_1_acc_27: 0.2188 - dense_1_acc_28: 0.3438 - dense_1_acc_29: 0.6250 - dense_1_acc_30: 0.2812 - dense_1_acc_31: 0.3125 - dense_1_acc_32: 0.0625 - dense_1_acc_33: 0.2344 - dense_1_acc_34: 0.5469 - dense_1_acc_35: 0.2656 - dense_1_acc_36: 0.4

973/973 [==============================] - 7s 7ms/step - loss: 382.0624 - dense_1_loss: 3.2595 - dense_1_acc: 0.1675 - dense_1_acc_1: 0.3864 - dense_1_acc_2: 0.4388 - dense_1_acc_3: 0.4430 - dense_1_acc_4: 0.4018 - dense_1_acc_5: 0.1398 - dense_1_acc_6: 0.1336 - dense_1_acc_7: 0.0185 - dense_1_acc_8: 0.1829 - dense_1_acc_9: 0.2600 - dense_1_acc_10: 0.2539 - dense_1_acc_11: 0.4080 - dense_1_acc_12: 0.4512 - dense_1_acc_13: 0.5087 - dense_1_acc_14: 0.3022 - dense_1_acc_15: 0.2354 - dense_1_acc_16: 0.2662 - dense_1_acc_17: 0.2138 - dense_1_acc_18: 0.2929 - dense_1_acc_19: 0.1850 - dense_1_acc_20: 0.2641 - dense_1_acc_21: 0.3823 - dense_1_acc_22: 0.3566 - dense_1_acc_23: 0.0216 - dense_1_acc_24: 0.3237 - dense_1_acc_25: 0.2785 - dense_1_acc_26: 0.2169 - dense_1_acc_27: 0.1829 - dense_1_acc_28: 0.3977 - dense_1_acc_29: 0.5190 - dense_1_acc_30: 0.3309 - dense_1_acc_31: 0.3392 - dense_1_acc_32: 0.1429 - dense_1_acc_33: 0.2302 - dense_1_acc_34: 0.4676 - dense_1_acc_35: 0.2939 - dense_1_acc_36:

 64/973 [>.............................] - ETA: 6s - loss: 381.6984 - dense_1_loss: 3.2862 - dense_1_acc: 0.2188 - dense_1_acc_1: 0.3750 - dense_1_acc_2: 0.4219 - dense_1_acc_3: 0.3750 - dense_1_acc_4: 0.3594 - dense_1_acc_5: 0.1250 - dense_1_acc_6: 0.1250 - dense_1_acc_7: 0.0156 - dense_1_acc_8: 0.1250 - dense_1_acc_9: 0.3125 - dense_1_acc_10: 0.2031 - dense_1_acc_11: 0.4531 - dense_1_acc_12: 0.4375 - dense_1_acc_13: 0.5312 - dense_1_acc_14: 0.2812 - dense_1_acc_15: 0.1875 - dense_1_acc_16: 0.4219 - dense_1_acc_17: 0.2188 - dense_1_acc_18: 0.2969 - dense_1_acc_19: 0.2344 - dense_1_acc_20: 0.2344 - dense_1_acc_21: 0.4688 - dense_1_acc_22: 0.3125 - dense_1_acc_23: 0.0000e+00 - dense_1_acc_24: 0.3281 - dense_1_acc_25: 0.2344 - dense_1_acc_26: 0.2031 - dense_1_acc_27: 0.1562 - dense_1_acc_28: 0.4062 - dense_1_acc_29: 0.4688 - dense_1_acc_30: 0.2812 - dense_1_acc_31: 0.3125 - dense_1_acc_32: 0.1250 - dense_1_acc_33: 0.2500 - dense_1_acc_34: 0.4844 - dense_1_acc_35: 0.2188 - dense_1_acc_36:

973/973 [==============================] - 7s 7ms/step - loss: 382.0602 - dense_1_loss: 3.2595 - dense_1_acc: 0.1675 - dense_1_acc_1: 0.3864 - dense_1_acc_2: 0.4388 - dense_1_acc_3: 0.4430 - dense_1_acc_4: 0.4018 - dense_1_acc_5: 0.1398 - dense_1_acc_6: 0.1336 - dense_1_acc_7: 0.0185 - dense_1_acc_8: 0.1829 - dense_1_acc_9: 0.2600 - dense_1_acc_10: 0.2539 - dense_1_acc_11: 0.4080 - dense_1_acc_12: 0.4512 - dense_1_acc_13: 0.5087 - dense_1_acc_14: 0.3022 - dense_1_acc_15: 0.2354 - dense_1_acc_16: 0.2662 - dense_1_acc_17: 0.2138 - dense_1_acc_18: 0.2929 - dense_1_acc_19: 0.1850 - dense_1_acc_20: 0.2641 - dense_1_acc_21: 0.3823 - dense_1_acc_22: 0.3566 - dense_1_acc_23: 0.0216 - dense_1_acc_24: 0.3237 - dense_1_acc_25: 0.2785 - dense_1_acc_26: 0.2169 - dense_1_acc_27: 0.1829 - dense_1_acc_28: 0.3977 - dense_1_acc_29: 0.5190 - dense_1_acc_30: 0.3309 - dense_1_acc_31: 0.3392 - dense_1_acc_32: 0.1429 - dense_1_acc_33: 0.2302 - dense_1_acc_34: 0.4676 - dense_1_acc_35: 0.2939 - dense_1_acc_36:

...

973/973 [==============================] - 7s 7ms/step - loss: 382.0031 - dense_1_loss: 3.2606 - dense_1_acc: 0.1675 - dense_1_acc_1: 0.3864 - dense_1_acc_2: 0.4388 - dense_1_acc_3: 0.4430 - dense_1_acc_4: 0.4018 - dense_1_acc_5: 0.1398 - dense_1_acc_6: 0.1336 - dense_1_acc_7: 0.0185 - dense_1_acc_8: 0.1829 - dense_1_acc_9: 0.2600 - dense_1_acc_10: 0.2539 - dense_1_acc_11: 0.4080 - dense_1_acc_12: 0.4512 - dense_1_acc_13: 0.5087 - dense_1_acc_14: 0.3022 - dense_1_acc_15: 0.2354 - dense_1_acc_16: 0.2662 - dense_1_acc_17: 0.2138 - dense_1_acc_18: 0.2929 - dense_1_acc_19: 0.1850 - dense_1_acc_20: 0.2641 - dense_1_acc_21: 0.3823 - dense_1_acc_22: 0.3566 - dense_1_acc_23: 0.0216 - dense_1_acc_24: 0.3237 - dense_1_acc_25: 0.2785 - dense_1_acc_26: 0.2169 - dense_1_acc_27: 0.1829 - dense_1_acc_28: 0.3977 - dense_1_acc_29: 0.5190 - dense_1_acc_30: 0.3309 - dense_1_acc_31: 0.3392 - dense_1_acc_32: 0.1429 - dense_1_acc_33: 0.2302 - dense_1_acc_34: 0.4676 - dense_1_acc_35: 0.2939 - dense_1_acc_36:

 64/973 [>.............................] - ETA: 6s - loss: 383.4071 - dense_1_loss: 3.2091 - dense_1_acc: 0.1250 - dense_1_acc_1: 0.4062 - dense_1_acc_2: 0.4844 - dense_1_acc_3: 0.3594 - dense_1_acc_4: 0.4688 - dense_1_acc_5: 0.1875 - dense_1_acc_6: 0.1094 - dense_1_acc_7: 0.0000e+00 - dense_1_acc_8: 0.1719 - dense_1_acc_9: 0.2500 - dense_1_acc_10: 0.2812 - dense_1_acc_11: 0.4062 - dense_1_acc_12: 0.5625 - dense_1_acc_13: 0.5156 - dense_1_acc_14: 0.3906 - dense_1_acc_15: 0.1719 - dense_1_acc_16: 0.1875 - dense_1_acc_17: 0.1562 - dense_1_acc_18: 0.2188 - dense_1_acc_19: 0.1875 - dense_1_acc_20: 0.3125 - dense_1_acc_21: 0.4844 - dense_1_acc_22: 0.3281 - dense_1_acc_23: 0.0469 - dense_1_acc_24: 0.3125 - dense_1_acc_25: 0.2812 - dense_1_acc_26: 0.2188 - dense_1_acc_27: 0.1875 - dense_1_acc_28: 0.5000 - dense_1_acc_29: 0.4219 - dense_1_acc_30: 0.3750 - dense_1_acc_31: 0.3594 - dense_1_acc_32: 0.0938 - dense_1_acc_33: 0.2031 - dense_1_acc_34: 0.4219 - dense_1_acc_35: 0.2188 - dense_1_acc_36:

973/973 [==============================] - 7s 7ms/step - loss: 382.0023 - dense_1_loss: 3.2618 - dense_1_acc: 0.1675 - dense_1_acc_1: 0.3864 - dense_1_acc_2: 0.4388 - dense_1_acc_3: 0.4430 - dense_1_acc_4: 0.4018 - dense_1_acc_5: 0.1398 - dense_1_acc_6: 0.1336 - dense_1_acc_7: 0.0185 - dense_1_acc_8: 0.1829 - dense_1_acc_9: 0.2600 - dense_1_acc_10: 0.2539 - dense_1_acc_11: 0.4080 - dense_1_acc_12: 0.4512 - dense_1_acc_13: 0.5087 - dense_1_acc_14: 0.3022 - dense_1_acc_15: 0.2354 - dense_1_acc_16: 0.2662 - dense_1_acc_17: 0.2138 - dense_1_acc_18: 0.2929 - dense_1_acc_19: 0.1850 - dense_1_acc_20: 0.2641 - dense_1_acc_21: 0.3823 - dense_1_acc_22: 0.3566 - dense_1_acc_23: 0.0216 - dense_1_acc_24: 0.3237 - dense_1_acc_25: 0.2785 - dense_1_acc_26: 0.2169 - dense_1_acc_27: 0.1829 - dense_1_acc_28: 0.3977 - dense_1_acc_29: 0.5190 - dense_1_acc_30: 0.3309 - dense_1_acc_31: 0.3392 - dense_1_acc_32: 0.1429 - dense_1_acc_33: 0.2302 - dense_1_acc_34: 0.4676 - dense_1_acc_35: 0.2939 - dense_1_acc_36:

 64/973 [>.............................] - ETA: 6s - loss: 382.1379 - dense_1_loss: 3.3753 - dense_1_acc: 0.1562 - dense_1_acc_1: 0.3906 - dense_1_acc_2: 0.4531 - dense_1_acc_3: 0.4688 - dense_1_acc_4: 0.2969 - dense_1_acc_5: 0.1406 - dense_1_acc_6: 0.1719 - dense_1_acc_7: 0.0000e+00 - dense_1_acc_8: 0.1875 - dense_1_acc_9: 0.2344 - dense_1_acc_10: 0.2188 - dense_1_acc_11: 0.3750 - dense_1_acc_12: 0.4531 - dense_1_acc_13: 0.5000 - dense_1_acc_14: 0.3906 - dense_1_acc_15: 0.2031 - dense_1_acc_16: 0.2812 - dense_1_acc_17: 0.1875 - dense_1_acc_18: 0.2344 - dense_1_acc_19: 0.1875 - dense_1_acc_20: 0.1094 - dense_1_acc_21: 0.3281 - dense_1_acc_22: 0.3125 - dense_1_acc_23: 0.0156 - dense_1_acc_24: 0.4062 - dense_1_acc_25: 0.2188 - dense_1_acc_26: 0.2188 - dense_1_acc_27: 0.1562 - dense_1_acc_28: 0.3750 - dense_1_acc_29: 0.5781 - dense_1_acc_30: 0.3750 - dense_1_acc_31: 0.3750 - dense_1_acc_32: 0.1406 - dense_1_acc_33: 0.2969 - dense_1_acc_34: 0.4844 - dense_1_acc_35: 0.3438 - dense_1_acc_36:

973/973 [==============================] - 7s 7ms/step - loss: 382.0010 - dense_1_loss: 3.2599 - dense_1_acc: 0.1675 - dense_1_acc_1: 0.3864 - dense_1_acc_2: 0.4388 - dense_1_acc_3: 0.4430 - dense_1_acc_4: 0.4018 - dense_1_acc_5: 0.1398 - dense_1_acc_6: 0.1336 - dense_1_acc_7: 0.0185 - dense_1_acc_8: 0.1829 - dense_1_acc_9: 0.2600 - dense_1_acc_10: 0.2539 - dense_1_acc_11: 0.4080 - dense_1_acc_12: 0.4512 - dense_1_acc_13: 0.5087 - dense_1_acc_14: 0.3022 - dense_1_acc_15: 0.2354 - dense_1_acc_16: 0.2662 - dense_1_acc_17: 0.2138 - dense_1_acc_18: 0.2929 - dense_1_acc_19: 0.1850 - dense_1_acc_20: 0.2641 - dense_1_acc_21: 0.3823 - dense_1_acc_22: 0.3566 - dense_1_acc_23: 0.0216 - dense_1_acc_24: 0.3237 - dense_1_acc_25: 0.2785 - dense_1_acc_26: 0.2169 - dense_1_acc_27: 0.1829 - dense_1_acc_28: 0.3977 - dense_1_acc_29: 0.5190 - dense_1_acc_30: 0.3309 - dense_1_acc_31: 0.3392 - dense_1_acc_32: 0.1429 - dense_1_acc_33: 0.2302 - dense_1_acc_34: 0.4676 - dense_1_acc_35: 0.2939 - dense_1_acc_36:

# Use Trained Weights for Text Generation

In [13]:
epochs_completed = 200
model.save('../data/trained-{}-epochs.h5'.format(epochs_completed))